In [1]:
import torch
import numpy as np

In [2]:
phone2index = {"sil": 0, "spn": 1, "sp": 2, "OW0": 3, "UW1": 4, "EY0": 5, "AW1": 6, "AW0": 7, 
               "AH0": 8, "AH2": 9, "EY1": 10, "AH1": 11, "AO1": 12, "AY2": 13, "EH2": 14, 
               "UW2": 15, "K": 16, "NG": 17, "F": 18, "JH": 19, "AY0": 20, "AO0": 21, "M": 22, 
               "CH": 23, "IH2": 24, "UH0": 25, "HH": 26, "L": 27, "AA2": 28, "IH0": 29, "R": 30, 
               "TH": 31, "AA1": 32, "AE2": 33, "D": 34, "Z": 35, "EH1": 36, "IH1": 37, "OW1": 38, 
               "OY2": 39, "DH": 40, "AE1": 41, "IY1": 42, "UH1": 43, "AW2": 44, "OY1": 45, "UH2": 46, 
               "UW0": 47, "B": 48, "W": 49, "AE0": 50, "OW2": 51, "S": 52, "EY2": 53, "AO2": 54, 
               "OY0": 55, "T": 56, "SH": 57, "ZH": 58, "IY0": 59, "AY1": 60, "EH0": 61, "ER1": 62,
               "V": 63, "IY2": 64, "Y": 65, "N": 66, "ER2": 67, "ER0": 68, "G": 69, "AA0": 70, 
               "P": 71}

phone2index_n = {"sil": 0, "OW": 1, "UW": 2, "EY": 3, "AW": 4, "AH": 5, "AO": 6, "AY": 7, "EH": 8, 
               "K": 9, "NG": 10, "F": 11, "JH": 12, "M": 13, "CH": 14, "IH": 15, "UH": 16, "HH": 17,
               "L": 18, "AA": 19, "R": 20, "TH": 21, "AE": 22, "D": 23, "Z": 24, "OY": 25, "DH": 26, 
               "IY": 27, "B": 28, "W": 29, "S": 30, "T": 31, "SH": 32, "ZH": 33, "ER": 34, "V": 35, 
               "Y": 36, "N": 37, "G": 38, "P": 39}

index2phone = {v: k for k, v in phone2index_n.items()}
keys_list = list(phone2index.keys())

In [3]:
# load reference articulatory for each phoneme
ema_ft = torch.load('/home/xuanru/art/art-pos/progress_16000.pt')
print(ema_ft.shape) # [40, 12]

torch.Size([40, 12])


In [4]:
ema_ft[7,:] = ema_ft[5,:] + ema_ft[27,:]  # AY = 0.4AH + 0.6IY
ema_ft[3,:] = ema_ft[8,:] + ema_ft[27,:]  # EY = 0.5EH + 0.5IY
ema_ft[1,:] = ema_ft[6,:] + ema_ft[2,:]   # OW = 0.6AO + 0.4UW
ema_ft[4,:] = ema_ft[19,:] + ema_ft[2,:]  # AW = 0.7AA + 0.3UW
ema_ft[25,:] = ema_ft[6,:] + ema_ft[27,:]  # OY = 0.4AO + 0.6IY

In [5]:
# define threshold
ema = ema_ft[1:] # w/o sil
print(ema.shape)

max_distance = float('-inf')
row_pair = None

for i in range(ema.size(0)):
    for j in range(i + 1, ema.size(0)):
        distance = torch.norm(ema[i] - ema[j], p=2).item()
        if distance > max_distance:
            max_distance = distance
            row_pair = (i, j)

print(f"max_d: {max_distance}")
print(f"row_piar: {row_pair}") # 2 28 

factor = 0.7 # adjust
threshold = max_distance * factor
print(threshold)

torch.Size([39, 12])


max_d: 7.3915696144104
row_piar: (2, 28)
5.17409873008728


In [6]:
ref_ema = ema_ft

### Frame-level

In [27]:
import sys
sys.path.append('..')
from model.encodec import ArticulatoryEncodec
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import numpy as np
import json
import torch

# dependency - check requirements
# download ckpts from https://drive.google.com/drive/u/1/folders/1XICRfrWo7G6EkYKRQafw24DwJh_AbHwz
# and place them under '../ckpts'

config_file = "../configs/ep2w_hifigan_libritts_eng.yml"
with open(config_file) as f:
     config = yaml.load(f, Loader=yaml.Loader)

config['linear_model_path']= Path.cwd().parent/"ckpts/wavlm_large-9_cut-10_mngu_linear.pkl"
config['generator_ckpt']= Path.cwd().parent/"ckpts/generator_ep2w_hifigan_multilang.ckpt"
config['spk_ft_ckpt'] = Path.cwd().parent/"ckpts/spkfc_ep2w_hifigan_multilang.ckpt"
config['device'] = 'cuda:0'
device = torch.device(config['device'])

encodec = ArticulatoryEncodec(**config)

Some weights of the model checkpoint at microsoft/wavlm-large were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at microsoft/wavlm-large and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [28]:
def cal_frame_per(wav_file, hypo_phn_index, ref_ema, threshold, device):
    code = encodec.encode(wav_file)
    ema = code['art'][:,:12] # [T, 12]
    ema = torch.tensor(ema).to(device)

    hypo_phn_art = ref_ema[hypo_phn_index].to(device)

    if hypo_phn_art.shape[0] == ema.shape[0]:
        T = ema.shape[0]
        print(ema.shape)
        print(hypo_phn_art.shape)
        distances = torch.norm(ema - hypo_phn_art, dim=1, p=2)
        neg_mask = distances > threshold

        neg_count = neg_mask.sum().item()
        neg_ratio = neg_count / T

        return neg_ratio
    
    return None

In [38]:
wav_file = "/data/xuanru/VCTK/VCTK_16k/p225/p225_001.wav"

# here I use GT_phn_idx for example, please replace hypo_phn_index with the phn list obtained from your decoding
mfa = np.load('/data/xuanru/VCTK/MFA_label_16k/p225/p225_001.npy')
mfa = np.vectorize(phone2index_n.get)([''.join(filter(str.isalpha, keys_list[val])) for val in mfa])
hypo_phn_index = mfa

aer = cal_frame_per(wav_file, hypo_phn_index, ref_ema, threshold, device)
print(f"AER: {aer * 100}%")

torch.Size([102, 12])
torch.Size([102, 12])
AER: 0.9803921568627451%


### Phn-level

In [7]:
def calculate_phn_aer(hypo_phn_list, gt_phn_list, ref_ema, threshold, device):
    errors = 0
    total_comparisons = len(gt_phn_list)
    
    for i in range(total_comparisons):
        if i < len(hypo_phn_list) and i < len(gt_phn_list):
            phn1 = hypo_phn_list[i]
            phn2 = gt_phn_list[i]
            art1 = ref_ema[phone2index_n[phn1]].to(device)
            art2 = ref_ema[phone2index_n[phn2]].to(device)
            
            distance = torch.norm(art1 - art2, p=2)
            print(distance)
            
            if distance > threshold:
                errors += 1 * threshold/distance
        else:
            errors += 1
    
    aer = errors / total_comparisons
    return aer

In [17]:
device = torch.device('cuda:1')

hypo_phn_list = ['P', 'L', 'EY', 'Z', 'K', 'K', 'AW', 'L']
gt_phn_list = ['P', 'L', 'IY', 'Z', 'K', 'AO', 'L']

threshold_phn = max_distance * 0.35
print(threshold_phn)

aer = calculate_phn_aer(hypo_phn_list, gt_phn_list, ref_ema, threshold_phn, device)
print(f"aer: {aer * 100}%")

2.58704936504364
tensor(0., device='cuda:1')
tensor(0., device='cuda:1')
tensor(2.5988, device='cuda:1')
tensor(0., device='cuda:1')
tensor(0., device='cuda:1')
tensor(6.1105, device='cuda:1')
tensor(2.1792, device='cuda:1')
aer: 20.26929473876953%
